# LEEP 2024 Multi-Country CEA

In [1]:
import numpy as np
import pandas as pd
import squigglepy as sq
# from squigglepy.numbers import K

pd.set_option('display.float_format', lambda x: f'{x:,.3f}')
idx = pd.IndexSlice

COUNTRIES = ["Malawi", "Madagascar", "Sierra Leone", "Liberia", "Angola", "Pakistan",
             "Niger", "Zimbabwe", "Bolivia", "Ghana", "Nigeria", "Cote d'Ivoire", "Senegal"]
SCENARIOS = ["Counterfactual", "LEEP intervention"]
YEARS = range(2021, 2101)

# N = 100*K
N = 10
SAMPLES = range(N)

SAMPLE_YEAR_INDEX = pd.MultiIndex.from_product(
    [SAMPLES, YEARS], names=["Sample", "Year"])
SCENARIO_SAMPLE_INDEX = pd.MultiIndex.from_product(
    [SCENARIOS, SAMPLES], names=["Scenario", "Sample"])
SCENARIO_SAMPLE_YEAR_INDEX = pd.MultiIndex.from_product(
    [SCENARIOS, SAMPLES, YEARS], names=["Scenario", "Sample", "Year"])

## Inputs

### Population

The population and projected future population estimates for each country are from the [UN population projections](https://population.un.org/wpp/Download/Standard/CSV/).

In [2]:
# TODO(drw): Could use uncertainty estimates here?
population = pd.read_csv('population.csv', index_col=0)
population.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Year,,,,,,,,,,,,,
2021,19889742,28915652,8420642,5193422,34503776,231402110,25252722,15993525,12079474,32833036,213401330,27478250,16876726
2022,20405318,29611718,8605723,5302690,35588996,235824860,26207982,16320539,12224114,33475870,218541220,28160548,17316452
2023,20931750,30325736,8791092,5418378,36684212,240485660,27202844,16665407,12388575,34121990,223804640,28873040,17763164
2024,21475962,31056608,8977979,5536954,37804630,245209810,28238976,17020320,12567344,34777524,229152220,29603304,18221560
2025,22033992,31797310,9165381,5657236,38942796,249948880,29315974,17375286,12746151,35439790,234573600,30344186,18687806


### Births

Projections of future annual births in each country are from the [UN population projections](https://population.un.org/wpp/Download/Standard/Population/).

In [3]:
# TODO(drw): Could use uncertainty estimates here?
births = pd.read_csv('births.csv', index_col=0)
births.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Year,,,,,,,,,,,,,
2021,653703,895045,264106,162685,1338792,6374741,1144371,488807,263820,904679,7923294,932943,549984
2022,665678,906130,265023,164452,1359723,6424944,1180654,491195,264070,906677,8002758,943133,555696
2023,678962,918152,266774,166979,1381742,6474370,1216927,493840,263936,909913,8109136,959962,562482
2024,687950,927190,267219,169153,1404185,6512176,1256001,496655,265235,911299,8234399,974998,568369
2025,698306,937701,268121,171399,1426882,6559877,1296167,498207,266191,916924,8321052,989981,575820


### Decline in lead market share once enforcement/compliance begins

We estimate that, once enforcement/compliance begins, the market share captured by lead-based paint will decline at a rate of 20% annually. This is a conservative estimate relative to our compliance goal: in the majority of our target countries, we aim to achieve 50% compliance a year after regulation is implemented and enforcement begins. [Follow up study](https://leadelimination.org/malawi-follow-up-study/) data from Malawi suggests that our compliance goals are achievable, as do preliminary reports of manufacturers representing >50% of market share switching to lead-free alternatives in Pakistan, Ghana, and Zimbabwe.

In [4]:
decline_in_lead_market_share_once_enforcement_compliance_begins = pd.Series(
    sq.norm(-0.3, -0.1) @ N)
decline_in_lead_market_share_once_enforcement_compliance_begins.head()

0   -0.199
1   -0.091
2   -0.178
3   -0.187
4   -0.198
dtype: float64

### % of new painted homes that use some colored solvent based lead paint

To develop this estimate, we surveyed members of the LEEP team to understand their best guess of the portion of homes that use some coloured solvent-based paints in the countries where they work. We averaged the estimates provided by LEEP’s team members, which gave us the estimate that coloured solvent-based paint is used in 80% of new painted homes in the LMICs included in this model. We are uncertain of this estimate given the difficulty of approximating this input across the variety of painting practices used in LMICs.

In [5]:
pct_of_new_painted_homes_that_use_some_colored_solvent_based_lead_paint = pd.Series(
    sq.lognorm(0.65, 0.9) @ N)
pct_of_new_painted_homes_that_use_some_colored_solvent_based_lead_paint.head()

0   0.627
1   0.910
2   0.843
3   0.742
4   0.684
dtype: float64

### Lead market share of colored solvent based paints (2021)

In [6]:
lead_market_share_of_colored_solvent_based_paints_2021 = pd.DataFrame(
    np.array(
        [sq.lognorm(0.60, 0.83) @ N,
         sq.lognorm(0.60, 0.83) @ N,
         sq.lognorm(0.32, 0.44) @ N,
         sq.lognorm(0.60, 0.83) @ N,
         sq.lognorm(0.48, 0.66) @ N,
         sq.lognorm(0.52, 0.72) @ N,
         sq.lognorm(0.32, 0.44) @ N,
         sq.lognorm(0.52, 0.72) @ N,
         sq.lognorm(0.32, 0.44) @ N,
         sq.lognorm(0.16, 0.22) @ N,
         sq.lognorm(0.68, 0.94) @ N,
         sq.lognorm(0.60, 0.83) @ N,
         sq.lognorm(0.60, 0.83) @ N]).T,
    columns=COUNTRIES)
lead_market_share_of_colored_solvent_based_paints_2021.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
0,0.605,0.787,0.422,0.659,0.539,0.646,0.420,0.572,0.381,0.219,0.742,0.756,0.601
1,0.786,0.661,0.390,0.688,0.565,0.627,0.349,0.509,0.362,0.183,0.859,0.722,0.691
2,0.764,0.637,0.364,0.716,0.597,0.576,0.395,0.544,0.369,0.194,0.724,0.613,0.558
3,0.753,0.651,0.444,0.727,0.505,0.586,0.387,0.592,0.375,0.190,0.749,0.775,0.603
4,0.592,0.654,0.339,0.685,0.714,0.570,0.374,0.581,0.345,0.185,0.762,0.821,0.740


### Annual time discount rate

We discount future costs and benefits at a rate of 4% per year, in line with [GiveWell](https://docs.google.com/spreadsheets/d/1B1fODKVbnGP4fejsZCVNvBm5zvI1jC7DhkaJpFk6zfo/edit#gid=1362437801). There is debate over the appropriate discount rate. [Haacker, Hallett, and Atun (2020)](https://academic.oup.com/heapol/article/35/1/107/5591528) found that 85% of 188 global health CEAs used a discount of 3% for both health and costs. Given the 100-year timeframe of this model, this is an important factor (e.g. a value of 1 in year 1 discounted by 4% per year would be 0.46 in year 20, 0.14 in year 50, and 0.06 by year 70; whereas using a discount rate of 3% would give a value of 0.55 in year 20, 0.23 in year 50, and 0.13 by year 70).

In [7]:
annual_time_discount_rate = pd.Series(sq.norm(0.03, 0.05) @ N)
annual_time_discount_rate.head()

0   0.049
1   0.038
2   0.034
3   0.049
4   0.040
dtype: float64

### Discount factor

In [8]:
discount_factor = pd.DataFrame(
    np.array(
        [1/(1 + annual_time_discount_rate)**(year - YEARS[0]) for year in YEARS]).T,
    columns=YEARS)
discount_factor.head()

,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,1.000,0.953,0.909,0.866,0.826,0.787,0.750,0.715,0.682,0.650,...,0.035,0.033,0.032,0.030,0.029,0.028,0.026,0.025,0.024,0.023
1,1.000,0.963,0.928,0.894,0.861,0.829,0.799,0.769,0.741,0.714,...,0.073,0.070,0.067,0.065,0.063,0.060,0.058,0.056,0.054,0.052
2,1.000,0.967,0.935,0.904,0.875,0.846,0.818,0.791,0.765,0.740,...,0.096,0.093,0.090,0.087,0.084,0.081,0.078,0.076,0.073,0.071
3,1.000,0.954,0.909,0.867,0.827,0.789,0.752,0.717,0.684,0.652,...,0.036,0.034,0.033,0.031,0.030,0.028,0.027,0.026,0.025,0.024
4,1.000,0.962,0.925,0.889,0.855,0.822,0.790,0.760,0.731,0.703,...,0.064,0.062,0.060,0.057,0.055,0.053,0.051,0.049,0.047,0.045


### Increase in BLL from living in a home with lead paint

In [9]:
increase_in_bll_from_living_in_a_home_with_lead_paint = pd.Series(
    sq.norm(0.7, 3.0) @ N)
increase_in_bll_from_living_in_a_home_with_lead_paint.head()

0   2.281
1   1.927
2   0.800
3   0.735
4   2.151
dtype: float64

### Implied % of childhood BLL caused by lead paint initially

In [10]:
# TODO(drw): This formula references non-inputs

### Probability of success

We calculate a probability of success for each country, which encapsulates our estimate of the likelihood that our intervention that country is likely to succeed in its implementation, and hence that its associated benefits will be realised. These figures were estimated by the LEEP team based on knowledge of each country and an assessment of current progress towards implementation in each country. Overall, our probability of success estimates are relatively high, since they mostly capture the “stochastic” possibility of failure due to events such as unexplained government withdrawal / reversal, or political instability, which we feel to be relatively unlikely. The possibility of implementation being delayed, on the other hand, is already captured in our uncertainty intervals around intervention start dates.

We multiply total discounted benefits by these probabilities of success to get total expected discounted benefits, and calculate the final cost-effectiveness number as total discounted costs divided by total expected discounted benefits. We choose only to weight benefits, and not costs, by the probabilities of success, in order to be conservative and reflect the possibility that failure occurs even despite government and LEEP costs having already been incurred.


In [11]:
probability_of_success = pd.DataFrame(
    np.array(
        [sq.lognorm(0.85, 0.95) @ N,
         sq.lognorm(0.75, 0.85) @ N,
         sq.lognorm(0.70, 0.80) @ N,
         sq.lognorm(0.65, 0.75) @ N,
         sq.lognorm(0.75, 0.85) @ N,
         sq.lognorm(0.85, 0.95) @ N,
         sq.lognorm(0.70, 0.80) @ N,
         sq.lognorm(0.75, 0.85) @ N,
         sq.lognorm(0.70, 0.80) @ N,
         sq.lognorm(0.75, 0.85) @ N,
         sq.lognorm(0.75, 0.85) @ N,
         sq.lognorm(0.70, 0.80) @ N,
         sq.lognorm(0.70, 0.80) @ N,]).T,
    columns=COUNTRIES)

probability_of_success.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
0,0.894,0.824,0.740,0.694,0.772,0.834,0.761,0.859,0.811,0.793,0.814,0.737,0.708
1,0.872,0.822,0.693,0.671,0.824,0.880,0.775,0.803,0.727,0.814,0.820,0.754,0.775
2,0.836,0.779,0.743,0.688,0.774,0.932,0.746,0.815,0.728,0.808,0.741,0.778,0.748
3,0.901,0.821,0.760,0.670,0.807,0.925,0.803,0.793,0.780,0.799,0.776,0.784,0.754
4,0.853,0.763,0.722,0.731,0.783,0.913,0.722,0.792,0.746,0.827,0.823,0.775,0.750


## Births with averted lead paint exposure

### Number of homes painted

#### People per household

We use recent data on average household size from the [Global Data Lab](https://globaldatalab.org/areadata/table/hhsize/AGO+CIV+GHA+LBR+MDG+MWI+NER+NGA+SEN+SLE+ZWE+BOL+PAK/?levels=1+4). We assume that average household size will decrease at a constant rate of 1% annually across countries. Average household size in high income countries is approximately 2.5, thus we treat this value as a long term minimum assumption in our estimates. Our estimate for people per household is used to calculate the total number of homes within a country. Our model is relatively insensitive to this input because the proportion of homes exposed to lead paint has a greater effect on our final estimate of cost-effectiveness than the absolute number of homes within the country.

In [12]:
PEOPLE_PER_HOUSEHOLD_INIT = pd.Series([3.96, 4.18, 9.01, 4.33, 6.45, 8.21, 7.45, 3.27, 4.86, 4.22, 6.86, 2.81, 10.8],
                                      index=COUNTRIES)
PEOPLE_PER_HOUSEHOLD_MIN = 2.5
PEOPLE_PER_HOUSEHOLD_RATE = -0.01

people_per_household = pd.DataFrame(
    np.array(
        [np.maximum(
            PEOPLE_PER_HOUSEHOLD_INIT * (1 + PEOPLE_PER_HOUSEHOLD_RATE)**(year - YEARS[0]), PEOPLE_PER_HOUSEHOLD_MIN)
         for year in YEARS]),
    index=YEARS, columns=COUNTRIES)
people_per_household.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
2021,3.960,4.180,9.010,4.330,6.450,8.210,7.450,3.270,4.860,4.220,6.860,2.810,10.800
2022,3.920,4.138,8.920,4.287,6.386,8.128,7.375,3.237,4.811,4.178,6.791,2.782,10.692
2023,3.881,4.097,8.831,4.244,6.322,8.047,7.302,3.205,4.763,4.136,6.723,2.754,10.585
2024,3.842,4.056,8.742,4.201,6.258,7.966,7.229,3.173,4.716,4.095,6.656,2.727,10.479
2025,3.804,4.015,8.655,4.159,6.196,7.886,7.156,3.141,4.668,4.054,6.590,2.699,10.374


#### Number of homes

In [13]:
number_of_homes = (population / people_per_household)
number_of_homes.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Year,,,,,,,,,,,,,
2021,"5,022,662.121","6,917,620.096","934,588.457","1,199,404.619","5,349,422.636","28,185,397.077","3,389,627.114","4,890,986.239","2,485,488.477","7,780,340.284","31,108,065.598","9,778,736.655","1,562,659.815"
2022,"5,204,907.152","7,155,700.063","964,777.968","1,237,009.821","5,573,407.877","29,014,242.301","3,553,383.771","5,041,404.566","2,540,656.358","8,012,798.602","32,179,111.818","10,122,775.082","1,619,570.894"
2023,"5,393,118.513","7,402,265.856","995,514.626","1,276,765.132","5,802,953.503","29,886,539.953","3,725,526.432","5,199,933.415","2,600,846.349","8,249,953.699","33,286,994.276","10,483,729.418","1,678,132.239"
2024,"5,589,228.400","7,657,238.157","1,026,947.425","1,317,884.758","6,040,594.647","30,781,451.838","3,906,493.336","5,364,316.387","2,665,027.225","8,493,381.351","34,426,618.705","10,857,461.081","1,738,826.363"
2025,"5,792,382.371","7,919,054.298","1,058,973.162","1,360,114.973","6,285,309.045","31,693,285.257","4,096,446.400","5,531,506.548","2,730,247.458","8,742,545.685","35,597,068.878","11,241,606.869","1,801,332.079"


#### Growth in the paint market

We use data from a [variety of sources](https://docs.google.com/spreadsheets/d/1bIiee05WlPt4B7NV3uvol2vu5JVbwOvL/edit#gid=508218074)  – IndustryArc, European Coatings, and Emerald Group, among others – to calculate an initial paint market growth rate of 5.5%. We arrive at this estimate by considering the average of the paint market growth rates from these sources, which is 5.75%. However, these growth rates correspond to a range of different countries and regions, and reflect growth in the paint market as a whole, rather than in the portion of the market that is specifically captured by home-use paints. Therefore, to be more conservative, we choose 5.5% as the initial value in 2021. 5.5% is also in line with a 20-year average of the World Bank’s estimate for real [GDP growth](https://data.worldbank.org/indicator/NY.GDP.MKTP.KD.ZG?locations=XO) in low- and middle-income countries (LMICs), which we would expect to approximate future GDP growth and correlate with paint market growth.

We assume that the paint market growth rate decreases by 0.03% per year (i.e. 5.5% in year 1, 5.47% in year 2). We expect that once all existing homes within a country are painted, then growth in the paint market will be roughly equal to population growth from that point forward. To reflect our expectation that growth in the paint market will converge with population growth in the long term, we treat the population growth rate in each country as the long term minimum for paint market growth.

This figure is a simplification in that we use one estimate of paint market growth rate for all countries within our model, rather than calculating unique growth rates for each country.


In [14]:
growth_in_the_paint_market = pd.Series(sq.norm(0.025, 0.08) @ N)
growth_in_the_paint_market.head()

0   0.031
1   0.027
2   0.078
3   0.055
4   0.038
dtype: float64

#### % population urban

We use UN population projections of urban populations to determine the proportion of each country’s population that lives in an urban area. We use the UN’s estimate for the urban percentage of the population in 2020.

In our Malawi CEA, we assumed that the proportion of homes that are painted is equal to the proportion of the population that lives in urban areas. In our current CEA, we update this starting assumption such that in Year 1, the proportion of homes that are painted is equal to the sum of 75% of the urban population and 25% of the rural (non-urban) population. In our model, the urban population percentage is only used to calculate the percentage of homes painted in Year 1 (2021). In future years, our estimate for the percentage of homes painted is calculated using a different formula that is primarily dependent on changes in the paint market growth rate, rather than urban population projections. We use this approach because we are uncertain about how urban-rural classification will correspond with home painting rates in future years. For example, it is possible that the proportion of homes painted in rural areas will increase significantly as countries continue to experience economic growth.


In [15]:
pct_population_urban = pd.DataFrame(
    np.array(
        [sq.lognorm(0.165, 0.183) @ N,
         sq.lognorm(0.371, 0.410) @ N,
         sq.lognorm(0.409, 0.452) @ N,
         sq.lognorm(0.494, 0.546) @ N,
         sq.lognorm(0.637, 0.704) @ N,
         sq.lognorm(0.353, 0.391) @ N,
         sq.lognorm(0.162, 0.179) @ N,
         sq.lognorm(0.304, 0.336) @ N,
         sq.lognorm(0.665, 0.735) @ N,
         sq.lognorm(0.542, 0.599) @ N,
         sq.lognorm(0.494, 0.546) @ N,
         sq.lognorm(0.494, 0.546) @ N,
         sq.lognorm(0.456, 0.504) @ N]).T,
    columns=COUNTRIES)
pct_population_urban.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
0,0.174,0.397,0.423,0.529,0.687,0.387,0.172,0.312,0.714,0.559,0.526,0.545,0.485
1,0.167,0.397,0.429,0.520,0.650,0.367,0.161,0.329,0.721,0.554,0.536,0.514,0.508
2,0.178,0.381,0.433,0.509,0.677,0.361,0.173,0.329,0.675,0.570,0.510,0.547,0.463
3,0.166,0.388,0.436,0.514,0.655,0.354,0.169,0.338,0.742,0.553,0.538,0.532,0.501
4,0.168,0.403,0.421,0.536,0.628,0.355,0.171,0.313,0.722,0.552,0.525,0.524,0.478


#### Number of painted homes

In [16]:
PCT_OF_HOMES_PAINTED_URBAN = 0.75
PCT_OF_HOMES_PAINTED_RURAL = 0.25
PCT_OF_HOMES_PAINTED_INIT = PCT_OF_HOMES_PAINTED_URBAN*pct_population_urban + \
    PCT_OF_HOMES_PAINTED_RURAL*(1 - pct_population_urban)

number_of_painted_homes = pd.DataFrame(
    index=SAMPLE_YEAR_INDEX, columns=COUNTRIES)
number_of_painted_homes.loc[idx[:, YEARS[0]], :] = \
    (number_of_homes.loc[YEARS[0]] * PCT_OF_HOMES_PAINTED_INIT).values

for year in YEARS[1:]:
    number_of_painted_homes.loc[idx[:, year], :] = np.minimum(
        number_of_painted_homes.loc[idx[:, year - 1], :].mul(
            1 + growth_in_the_paint_market, level=0, axis=0).values,
        number_of_homes.loc[year].values)

number_of_painted_homes.head()
# number_of_painted_homes.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi    Madagascar Sierra Leone     Liberia  \
Sample Year                                                        
0      2021 1,693,518.804 3,103,990.560  431,533.294 617,092.464   
       2022 1,746,555.295 3,201,199.263  445,047.765 636,418.153   
       2023 1,801,252.748 3,301,452.284  458,985.474 656,349.072   
       2024 1,857,663.179 3,404,844.963  473,359.675 676,904.174   
       2025 1,915,840.234 3,511,475.624  488,184.037 698,103.007   

                   Angola       Pakistan         Niger      Zimbabwe  \
Sample Year                                                            
0      2021 3,175,575.448 12,504,775.594 1,138,164.042 1,985,755.036   
       2022 3,275,025.999 12,896,391.807 1,173,808.303 2,047,943.587   
       2023 3,377,591.077 13,300,272.394 1,210,568.848 2,112,079.716   
       2024 3,483,368.219 13,716,801.444 1,248,480.635 2,178,224.419   
       2025 3,592,458.019 14,146,375.073 1,287,579.718 2,246,440.597   

                  Bolivia         Ghana        Nigeria Cote d'Ivoire  \
Sample Year                                                            
0      2021 1,508,917.649 4,118,972.587 15,962,331.629 5,111,339.815   
       2022 1,556,172.925 4,247,967.820 16,462,229.273 5,271,413.342   
       2023 1,604,908.110 4,381,002.840 16,977,782.377 5,436,499.945   
       2024 1,655,169.550 4,518,204.162 17,509,481.229 5,606,756.621   
       2025 1,707,005.045 4,659,702.263 18,057,831.471 5,782,345.281   

                Senegal  
Sample Year              
0      2021 769,299.950  
       2022 793,392.373  
       2023 818,239.305  
       2024 843,864.377  
       2025 870,291.958

#### % of homes painted

In [17]:
pct_of_homes_painted = number_of_painted_homes / number_of_homes
pct_of_homes_painted.head()
# pct_of_homes_painted.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi Madagascar Sierra Leone Liberia Angola Pakistan Niger  \
Sample Year                                                                
0      2021  0.337      0.449        0.462   0.514  0.594    0.444 0.336   
       2022  0.336      0.447        0.461   0.514  0.588    0.444 0.330   
       2023  0.334      0.446        0.461   0.514  0.582    0.445 0.325   
       2024  0.332      0.445        0.461   0.514  0.577    0.446 0.320   
       2025  0.331      0.443        0.461   0.513  0.572    0.446 0.314   

            Zimbabwe Bolivia Ghana Nigeria Cote d'Ivoire Senegal  
Sample Year                                                       
0      2021    0.406   0.607 0.529   0.513         0.523   0.492  
       2022    0.406   0.613 0.530   0.512         0.521   0.490  
       2023    0.406   0.617 0.531   0.510         0.519   0.488  
       2024    0.406   0.621 0.532   0.509         0.516   0.485  
       2025    0.406   0.625 0.533   0.507         0.514   0.483

#### New painted homes

In [18]:
# TODO(drw): I'm a little skeptical that new paint will grow exponentially and suddenly hit a ceiling
new_painted_homes = number_of_painted_homes.groupby("Sample").diff()
new_painted_homes.head()
# new_painted_homes.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi  Madagascar Sierra Leone    Liberia      Angola  \
Sample Year                                                              
0      2021        NaN         NaN          NaN        NaN         NaN   
       2022 53,036.491  97,208.704   13,514.472 19,325.690  99,450.551   
       2023 54,697.453 100,253.021   13,937.709 19,930.919 102,565.077   
       2024 56,410.431 103,392.678   14,374.201 20,555.102 105,777.142   
       2025 58,177.055 106,630.661   14,824.362 21,198.833 109,089.800   

               Pakistan      Niger   Zimbabwe    Bolivia       Ghana  \
Sample Year                                                            
0      2021         NaN        NaN        NaN        NaN         NaN   
       2022 391,616.212 35,644.261 62,188.550 47,255.275 128,995.233   
       2023 403,880.588 36,760.545 64,136.130 48,735.185 133,035.020   
       2024 416,529.050 37,911.787 66,144.702 50,261.441 137,201.322   
       2025 429,573.629 39,099.083 68,216.178 51,835.495 141,498.101   

                Nigeria Cote d'Ivoire    Senegal  
Sample Year                                       
0      2021         NaN           NaN        NaN  
       2022 499,897.644   160,073.527 24,092.422  
       2023 515,553.104   165,086.603 24,846.933  
       2024 531,698.852   170,256.675 25,625.072  
       2025 548,350.242   175,588.660 26,427.581

#### % of prior year painted homes repainted

In our [first cost-effectiveness analysis](https://my.causal.app/models/61262) evaluating LEEP’s Malawi program, we used a conservative estimate for the repainting rate (2%) due to the relative lack of information available to inform this approximation. Since the original Malawi CEA was published, we have conducted informal surveys within our program countries to gather more information about repainting rates from relevant stakeholders. Data from these surveys suggest that repainting actually occurs about every 5-20 years within focus countries – implying a repainting rate within the range of 5-20%. In this CEA, we estimate the repainting rate at 4%. This is still a conservative estimate. In the future, we hope to conduct more formal data collection procedures that will allow us to further hone this estimate.

In [19]:
pct_of_prior_year_painted_homes_repainted = pd.Series(
    sq.lognorm(0.02, 0.1) @ N)
pct_of_prior_year_painted_homes_repainted.head()

0   0.059
1   0.089
2   0.034
3   0.058
4   0.057
dtype: float64

#### Number of homes repainted

In [20]:
number_of_homes_repainted = pd.DataFrame(
    index=SAMPLE_YEAR_INDEX, columns=COUNTRIES)

for year in YEARS[1:]:
    number_of_homes_repainted.loc[idx[:, year], :] = \
        number_of_painted_homes.loc[idx[:, year], :].mul(
        pct_of_prior_year_painted_homes_repainted, level=0, axis=0).values

number_of_homes_repainted.head()
# number_of_homes_repainted.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi  Madagascar Sierra Leone    Liberia      Angola  \
Sample Year                                                               
0      2021         NaN         NaN          NaN        NaN         NaN   
       2022 102,289.814 187,483.372   26,064.937 37,272.850 191,807.153   
       2023 105,493.258 193,354.851   26,881.221 38,440.137 197,814.042   
       2024 108,797.025 199,410.209   27,723.069 39,643.979 204,009.050   
       2025 112,204.258 205,655.205   28,591.282 40,885.523 210,398.069   

               Pakistan      Niger    Zimbabwe    Bolivia       Ghana  \
Sample Year                                                             
0      2021         NaN        NaN         NaN        NaN         NaN   
       2022 755,297.882 68,745.967 119,941.103 91,139.765 248,789.053   
       2023 778,951.797 70,898.907 123,697.338 93,994.019 256,580.462   
       2024 803,346.489 73,119.272 127,571.209 96,937.661 264,615.878   
       2025 828,505.158 75,409.172 131,566.399 99,973.490 272,902.941   

                  Nigeria Cote d'Ivoire    Senegal  
Sample Year                                         
0      2021           NaN           NaN        NaN  
       2022   964,136.876   308,728.782 46,466.298  
       2023   994,331.071   318,397.344 47,921.498  
       2024 1,025,470.867   328,368.699 49,422.271  
       2025 1,057,585.879   338,652.331 50,970.045

### Scenarios

#### Start of enforcement/compliance

The predicted start dates for enforcement/compliance in the counterfactual v.s. LEEP scenarios reflect a best guess estimate for the number of years that compliance to regulation is brought forward due to LEEP’s activities.

Our estimate for the year in which enforcement/compliance begins in the counterfactual scenario is based on our understanding of how much progress a country has made toward implementing lead paint regulation, as well as any barriers or catalysts identified, independent of LEEP intervention. We expect that enforcement of lead paint regulation will be initiated more quickly in countries where governments are already aware of the threats of lead poisoning and/or have already drafted regulation that bans lead paint manufacturing. For example, in Pakistan, where implementation has already begun, the start date for enforcement/compliance under the LEEP scenario is 2023 versus 2028 in the counterfactual scenario. In Angola, the start date is 2025 under the LEEP scenario and 2032 under the counterfactual scenario.

Historically, LMIC governments have very rarely implemented lead paint regulation without externally funded paint studies and advocacy. Of 38 LMICs with lead paint regulation, we were only able to identify one in Africa (Algeria), five in Latin America (all upper-middle income countries), and five in Europe (as a result of EU laws) that implemented lead paint regulation without an externally )funded paint study and advocacy. At present, support for progress toward lead paint regulation in LMICs is very limited, in part because the SAICM’s [Global Environment Facility Project](https://www.saicm.org/Implementation/GEFProject/tabid/7893/language/en-US/Default.aspx) has come to an end.
Under the LEEP scenario, we assume countries that have made more progress toward implementing lead paint regulation prior to intervention will achieve compliance objectives sooner with LEEP’s support than those that have made less regulatory progress. Our estimate for the LEEP scenario is also informed by observed progress within a given country since LEEP started working there. 


In [21]:
start_of_enforcement_compliance = pd.Series([2031, 2024], index=SCENARIOS)
start_of_enforcement_compliance.head()

Counterfactual       2031
LEEP intervention    2024
dtype: int64

#### Lead market share of colored solvent based paint

For each country, we estimate the proportion of paint within the coloured solvent-based paint market that has a lead concentration greater than 90 parts per million (ppm). Estimates are primarily based on studies conducted by LEEP and by [IPEN](https://ipen.org/) that evaluate lead content among paints produced and sold within a given country. We use domestic and regional industry data to inform our estimates for countries in which a formal paint study has not been conducted. We adjust estimates slightly to reflect any information available about the relative market share of different brands within a given country. It is possible that we overestimate the value of this input because the paint studies that inform this estimate primarily test yellow and red paints, which are more likely to contain lead than other coloured paints, such as blue paint.

At the same time, because we only consider the contribution of the coloured solvent-based paint market to overall lead paint exposure, our approach is also at risk of underestimating the overall number of homes that contain lead paint. For simplicity, we ignore the contribution of both white paints and water-based paints, which on average are less likely to contain high levels of lead. However, evidence indicates that these types of paint may still, in some cases, contain high levels of lead. Across studies of solvent-based paints in the countries included in this CEA, 26% of white paints had lead content >90 ppm. In a study of water-based paints [in Nigeria](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6221508/), all tested samples had lead content >90 ppm. LEEP typically does not test the lead content of water-based paints, but has also identified high lead levels in some countries. 

In [22]:
lead_market_share_of_colored_solvent_based_paint = pd.DataFrame(
    index=SCENARIO_SAMPLE_YEAR_INDEX, columns=COUNTRIES)

for scenario in SCENARIOS:
    lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario, :, YEARS[0]], :] = \
        lead_market_share_of_colored_solvent_based_paints_2021.values

    for year in YEARS[1:]:
        if year < start_of_enforcement_compliance[scenario]:
            lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario, :, year], :] = \
                lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario,
                                                                         :, year - 1], :].values
        else:
            lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario, :, year], :] = \
                lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario, :, year - 1], :].mul(
                (1 + decline_in_lead_market_share_once_enforcement_compliance_begins), level=1, axis=0).values

lead_market_share_of_colored_solvent_based_paint.head()
# lead_market_share_of_colored_solvent_based_paint.loc[idx['Counterfactual', :, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi Madagascar Sierra Leone Liberia Angola  \
Scenario       Sample Year                                                 
Counterfactual 0      2021  0.605      0.787        0.422   0.659  0.539   
                      2022  0.605      0.787        0.422   0.659  0.539   
                      2023  0.605      0.787        0.422   0.659  0.539   
                      2024  0.605      0.787        0.422   0.659  0.539   
                      2025  0.605      0.787        0.422   0.659  0.539   

                           Pakistan Niger Zimbabwe Bolivia Ghana Nigeria  \
Scenario       Sample Year                                                 
Counterfactual 0      2021    0.646 0.420    0.572   0.381 0.219   0.742   
                      2022    0.646 0.420    0.572   0.381 0.219   0.742   
                      2023    0.646 0.420    0.572   0.381 0.219   0.742   
                      2024    0.646 0.420    0.572   0.381 0.219   0.742   
                      2025    0.646 0.420    0.572   0.381 0.219   0.742   

                           Cote d'Ivoire Senegal  
Scenario       Sample Year                        
Counterfactual 0      2021         0.756   0.601  
                      2022         0.756   0.601  
                      2023         0.756   0.601  
                      2024         0.756   0.601  
                      2025         0.756   0.601

#### Number of homes with lead paint

In [23]:
# TODO(drw): Verify that % of new painted homes is used here as an approx. of lead paint in existing homes
PCT_OF_PAINTED_HOMES_WITH_LEAD_PAINT_INIT = pd.DataFrame(
    index=SCENARIO_SAMPLE_INDEX, columns=COUNTRIES)
for scenario in SCENARIOS:
    PCT_OF_PAINTED_HOMES_WITH_LEAD_PAINT_INIT.loc[idx[scenario, :], :] = \
        lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario, :, YEARS[0]], :].mul(
        pct_of_new_painted_homes_that_use_some_colored_solvent_based_lead_paint, level=1, axis=0).values

number_of_homes_with_lead_paint = pd.DataFrame(
    index=SCENARIO_SAMPLE_YEAR_INDEX, columns=COUNTRIES)

for scenario in SCENARIOS:
    number_of_homes_with_lead_paint.loc[idx[scenario, :, YEARS[0]], :] = \
        PCT_OF_PAINTED_HOMES_WITH_LEAD_PAINT_INIT.loc[idx[scenario, :], :] * \
        number_of_painted_homes.loc[idx[:, YEARS[0]], :]

    for year in YEARS[1:]:
        number_of_homes_with_lead_paint.loc[idx[scenario, :, year], :] = \
            number_of_homes_with_lead_paint.loc[idx[scenario, :, year - 1], :].mul(
            (1 - pct_of_prior_year_painted_homes_repainted), level=1, axis=0).values + \
            (lead_market_share_of_colored_solvent_based_paint.loc[idx[scenario, :, year], :] *
             (number_of_homes_repainted.loc[idx[:, year], :] + new_painted_homes.loc[idx[:, year], :])).mul(
            pct_of_new_painted_homes_that_use_some_colored_solvent_based_lead_paint, level=1, axis=0).values

number_of_homes_with_lead_paint.head()
# number_of_homes_with_lead_paint.loc[idx['LEEP intervention', :, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi    Madagascar Sierra Leone     Liberia  \
Scenario       Sample Year                                                      
Counterfactual 0      2021 641,548.285 1,530,151.799  114,182.634 254,842.663   
                      2022 662,816.561 1,580,878.599  117,967.957 263,291.075   
                      2023 684,681.988 1,633,029.658  121,859.561 271,976.694   
                      2024 707,167.303 1,686,659.207  125,861.493 280,908.551   
                      2025 730,295.719 1,741,822.611  129,977.884 290,095.867   

                                  Angola      Pakistan       Niger  \
Scenario       Sample Year                                           
Counterfactual 0      2021 1,072,064.245 5,064,590.029 299,865.801   
                      2022 1,107,604.764 5,232,488.695 309,806.796   
                      2023 1,144,143.156 5,405,101.461 320,026.906   
                      2024 1,181,717.416 5,582,607.823 330,536.757   
                      2025 1,220,366.335 5,765,191.032 341,347.200   

                              Zimbabwe     Bolivia       Ghana       Nigeria  \
Scenario       Sample Year                                                     
Counterfactual 0      2021 711,766.621 360,602.872 564,947.542 7,420,604.983   
                      2022 735,362.739 372,557.392 583,676.391 7,666,609.037   
                      2023 759,621.367 384,847.560 602,931.090 7,919,520.160   
                      2024 784,567.731 397,486.155 622,731.663 8,179,601.349   
                      2025 810,227.583 410,486.226 643,098.550 8,447,121.101   

                           Cote d'Ivoire     Senegal  
Scenario       Sample Year                            
Counterfactual 0      2021 2,422,731.391 289,478.579  
                      2022 2,503,048.528 299,075.223  
                      2023 2,585,620.733 308,941.312  
                      2024 2,670,533.872 319,087.107  
                      2025 2,757,875.605 329,523.080

#### % of total homes with lead paint

In [24]:
pct_of_total_homes_with_lead_paint = number_of_homes_with_lead_paint/number_of_homes

pct_of_total_homes_with_lead_paint.head()
# pct_of_total_homes_with_lead_paint.loc[idx['Counterfactual', :, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi Madagascar Sierra Leone Liberia Angola  \
Scenario       Sample Year                                                 
Counterfactual 0      2021  0.128      0.221        0.122   0.212  0.200   
                      2022  0.127      0.221        0.122   0.213  0.199   
                      2023  0.127      0.221        0.122   0.213  0.197   
                      2024  0.127      0.220        0.123   0.213  0.196   
                      2025  0.126      0.220        0.123   0.213  0.194   

                           Pakistan Niger Zimbabwe Bolivia Ghana Nigeria  \
Scenario       Sample Year                                                 
Counterfactual 0      2021    0.180 0.088    0.146   0.145 0.073   0.239   
                      2022    0.180 0.087    0.146   0.147 0.073   0.238   
                      2023    0.181 0.086    0.146   0.148 0.073   0.238   
                      2024    0.181 0.085    0.146   0.149 0.073   0.238   
                      2025    0.182 0.083    0.146   0.150 0.074   0.237   

                           Cote d'Ivoire Senegal  
Scenario       Sample Year                        
Counterfactual 0      2021         0.248   0.185  
                      2022         0.247   0.185  
                      2023         0.247   0.184  
                      2024         0.246   0.184  
                      2025         0.245   0.183

#### Births in homes with lead paint

In [25]:
births_in_homes_with_lead_paint_counterfactual = pct_of_total_homes_with_lead_paint*births

births_in_homes_with_lead_paint_counterfactual.head()
# births_in_homes_with_lead_paint_counterfactual.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi  Madagascar Sierra Leone    Liberia  \
Scenario       Sample Year                                                  
Counterfactual 0      2021 83,497.960 197,980.620   32,266.950 34,566.382   
                      2022 84,770.466 200,187.475   32,405.613 35,002.749   
                      2023 86,197.448 202,555.471   32,655.434 35,569.891   
                      2024 87,041.665 204,232.064   32,750.053 36,055.144   
                      2025 88,041.474 206,250.487   32,909.050 36,557.308   

                                Angola      Pakistan       Niger   Zimbabwe  \
Scenario       Sample Year                                                    
Counterfactual 0      2021 268,303.915 1,145,467.265 101,237.604 71,134.223   
                      2022 270,218.097 1,158,687.740 102,936.991 71,647.989   
                      2023 272,432.073 1,170,915.964 104,535.396 72,141.581   
                      2024 274,699.755 1,181,065.951 106,272.931 72,639.169   
                      2025 277,045.845 1,193,279.388 108,006.534 72,974.885   

                              Bolivia      Ghana       Nigeria Cote d'Ivoire  \
Scenario       Sample Year                                                     
Counterfactual 0      2021 38,275.876 65,690.723 1,890,044.714   231,141.340   
                      2022 38,722.762 66,045.084 1,906,641.089   233,207.559   
                      2023 39,054.643 66,499.141 1,929,296.034   236,757.126   
                      2024 39,559.536 66,816.115 1,956,454.154   239,813.448   
                      2025 40,021.185 67,448.603 1,974,570.832   242,869.590   

                               Senegal  
Scenario       Sample Year              
Counterfactual 0      2021 101,883.075  
                      2022 102,616.629  
                      2023 103,551.987  
                      2024 104,299.787  
                      2025 105,336.480

### Births with averted lead paint exposure vs. counterfactual

In [26]:
births_with_averted_lead_paint_exposure_vs_counterfactual = \
    births_in_homes_with_lead_paint_counterfactual.loc[idx['Counterfactual', :, :], :].droplevel(0) - \
    births_in_homes_with_lead_paint_counterfactual.loc[idx['LEEP intervention', :, :], :].droplevel(
        0)

births_with_averted_lead_paint_exposure_vs_counterfactual.head()
# births_with_averted_lead_paint_exposure_vs_counterfactual.loc[idx[:, :], 'Pakistan'].unstack(level=-1).T.plot()

Malawi Madagascar Sierra Leone   Liberia     Angola   Pakistan  \
Sample Year                                                                     
0      2021     0.000      0.000        0.000     0.000      0.000      0.000   
       2022     0.000      0.000        0.000     0.000      0.000      0.000   
       2023     0.000      0.000        0.000     0.000      0.000      0.000   
       2024 1,535.110  3,601.937      577.596   635.886  4,844.740 20,829.860   
       2025 4,207.803  9,857.416    1,572.836 1,747.199 13,240.969 57,030.904   

                Niger  Zimbabwe   Bolivia     Ghana    Nigeria Cote d'Ivoire  \
Sample Year                                                                    
0      2021     0.000     0.000     0.000     0.000      0.000         0.000   
       2022     0.000     0.000     0.000     0.000      0.000         0.000   
       2023     0.000     0.000     0.000     0.000      0.000         0.000   
       2024 1,874.282 1,281.100   697.691 1,178.402 34,504.987     4,229.468   
       2025 5,162.002 3,487.719 1,912.749 3,223.599 94,371.495    11,607.569   

              Senegal  
Sample Year            
0      2021     0.000  
       2022     0.000  
       2023     0.000  
       2024 1,839.482  
       2025 5,034.391

### Total number of births with averted lead paint exposure

In [27]:
total_number_of_births_with_averted_lead_paint_exposure = \
    births_with_averted_lead_paint_exposure_vs_counterfactual.groupby(
        level=0).sum()

total_number_of_births_with_averted_lead_paint_exposure.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Sample,,,,,,,,,,,,,
0,"703,196.706","1,690,005.166","255,546.910","301,174.946","2,236,898.329","9,794,461.941","891,502.390","623,629.942","328,468.743","576,506.618","15,624,177.937","2,321,430.220","890,529.041"
1,"1,225,695.781","1,932,731.617","320,694.648","423,343.395","3,073,055.470","12,584,137.668","983,771.599","778,526.769","426,439.727","657,445.605","24,608,588.842","3,017,142.790","1,429,982.891"
2,"3,325,329.303","4,772,359.978","743,411.927","1,062,983.675","8,028,661.202","29,115,172.870","3,158,698.352","2,194,803.963","809,825.784","1,615,521.571","49,584,367.177","6,392,002.908","2,836,674.342"
3,"1,631,542.968","2,609,216.309","509,656.641","613,426.291","3,832,682.892","16,071,306.838","1,541,209.902","1,254,710.283","580,262.651","930,104.709","29,875,360.373","4,410,425.902","1,716,058.746"
4,"859,213.309","1,780,256.102","257,032.301","396,751.901","3,544,144.341","10,478,836.343","998,353.479","800,244.625","376,361.291","611,119.201","20,147,283.868","3,121,418.249","1,372,155.898"


### Total expected number of births with averted lead paint exposure

In [28]:
total_expected_number_of_births_with_averted_lead_paint_exposure = \
    total_number_of_births_with_averted_lead_paint_exposure * \
    probability_of_success

total_expected_number_of_births_with_averted_lead_paint_exposure.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Sample,,,,,,,,,,,,,
0,"628,659.680","1,393,369.734","189,210.554","209,097.215","1,725,951.489","8,173,387.244","678,663.723","535,637.300","266,229.280","457,182.185","12,712,697.641","1,711,258.300","630,454.464"
1,"1,069,162.263","1,588,536.065","222,346.870","284,130.707","2,531,057.225","11,069,103.409","761,992.823","624,783.324","310,074.272","535,297.513","20,189,496.391","2,274,481.256","1,108,487.253"
2,"2,779,311.412","3,716,705.422","552,249.276","730,935.132","6,216,469.071","27,125,791.165","2,355,744.867","1,789,574.457","589,276.386","1,305,290.767","36,719,014.376","4,972,369.781","2,123,025.028"
3,"1,470,077.255","2,141,442.616","387,213.496","411,157.858","3,093,590.372","14,859,494.596","1,237,458.672","994,689.368","452,585.531","743,311.758","23,197,506.761","3,456,155.560","1,294,673.659"
4,"732,553.983","1,358,626.067","185,600.057","289,996.087","2,775,667.013","9,566,213.886","720,608.530","633,513.025","280,726.231","505,212.551","16,580,623.881","2,418,642.076","1,028,535.267"


### Number of homes with averted lead paint vs. counterfactual

In [29]:
number_of_homes_with_averted_lead_paint_vs_counterfactual = \
    number_of_homes_with_lead_paint.loc[idx['Counterfactual', :, :], :].droplevel(0) - \
    number_of_homes_with_lead_paint.loc[idx['LEEP intervention', :, :], :].droplevel(
        0)

number_of_homes_with_averted_lead_paint_vs_counterfactual.head()

Malawi Madagascar Sierra Leone    Liberia     Angola  \
Sample Year                                                            
0      2021      0.000      0.000        0.000      0.000      0.000   
       2022      0.000      0.000        0.000      0.000      0.000   
       2023      0.000      0.000        0.000      0.000      0.000   
       2024 12,471.950 29,746.751    2,219.755  4,954.241 20,841.349   
       2025 34,903.331 83,247.661    6,212.088 13,864.674 58,325.482   

               Pakistan      Niger   Zimbabwe    Bolivia      Ghana  \
Sample Year                                                           
0      2021       0.000      0.000      0.000      0.000      0.000   
       2022       0.000      0.000      0.000      0.000      0.000   
       2023       0.000      0.000      0.000      0.000      0.000   
       2024  98,457.616  5,829.509 13,837.022  7,010.261 10,982.802   
       2025 275,538.202 16,314.150 38,723.548 19,618.541 30,735.880   

                Nigeria Cote d'Ivoire    Senegal  
Sample Year                                       
0      2021       0.000         0.000      0.000  
       2022       0.000         0.000      0.000  
       2023       0.000         0.000      0.000  
       2024 144,259.471    47,098.848  5,627.577  
       2025 403,716.814   131,808.310 15,749.035

### Total number of homes with averted lead paint

In [30]:
total_number_of_homes_with_averted_lead_paint = \
    number_of_homes_with_averted_lead_paint_vs_counterfactual.groupby(
        level=0).sum()

total_number_of_homes_with_averted_lead_paint.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Sample,,,,,,,,,,,,,
0,"9,248,737.146","22,059,090.327","1,646,088.006","3,673,879.794","15,455,174.061","73,012,524.680","4,322,948.147","10,260,990.612","5,197,699.935","8,144,403.610","106,977,488.877","34,926,459.060","4,173,203.091"
1,"17,306,641.097","26,952,401.209","2,225,974.601","5,532,212.443","22,834,392.327","100,620,505.988","5,131,222.900","13,549,868.259","7,182,979.801","9,879,474.101","181,871,585.570","47,041,013.293","7,150,929.697"
2,"50,770,495.193","70,901,735.963","5,495,925.752","14,685,553.468","63,616,213.682","248,346,199.933","18,241,897.871","40,064,216.349","14,144,399.988","25,376,357.714","391,360,341.483","102,618,748.827","15,161,837.780"
3,"22,039,085.012","34,858,349.298","3,361,281.309","7,645,133.070","27,136,404.199","122,615,675.063","7,681,310.831","21,039,795.006","9,221,879.514","13,356,011.821","209,519,414.328","66,995,544.028","8,224,169.360"
4,"11,434,118.482","23,494,835.057","1,676,329.653","4,895,398.403","24,786,947.505","79,033,522.887","4,900,645.216","13,289,991.108","6,013,386.086","8,722,522.276","139,710,022.853","47,249,099.448","6,501,020.743"


### Total expected number of homes with averted lead paint

In [31]:
total_expected_number_of_homes_with_averted_lead_paint = \
    total_number_of_homes_with_averted_lead_paint * \
    probability_of_success

total_expected_number_of_homes_with_averted_lead_paint.head()

,Malawi,Madagascar,Sierra Leone,Liberia,Angola,Pakistan,Niger,Zimbabwe,Bolivia,Ghana,Nigeria,Cote d'Ivoire,Senegal
Sample,,,,,,,,,,,,,
0,"8,268,395.016","18,187,204.061","1,218,786.887","2,550,670.439","11,924,941.040","60,928,271.655","3,290,880.784","8,813,190.219","4,212,820.664","6,458,687.757","87,042,817.610","25,746,280.213","2,954,439.888"
1,"15,096,411.235","22,152,512.533","1,543,332.539","3,712,993.883","18,807,064.901","88,506,563.990","3,974,454.055","10,874,040.652","5,222,912.157","8,043,947.487","149,211,957.830","35,461,995.151","5,543,223.258"
2,"42,433,997.911","55,218,145.262","4,082,690.778","10,098,167.284","49,257,057.289","231,377,199.305","13,604,736.030","32,667,108.059","10,292,288.882","20,503,301.234","289,816,464.810","79,827,617.890","11,347,429.134"
3,"19,857,986.114","28,609,032.707","2,553,745.765","5,124,261.189","21,903,434.519","113,370,180.745","6,167,430.333","16,679,595.819","7,192,758.711","10,673,723.639","162,686,841.922","52,499,923.396","6,204,691.688"
4,"9,748,579.254","17,930,395.127","1,210,458.289","3,578,171.592","19,412,390.109","72,150,338.012","3,537,270.936","10,521,010.959","4,485,358.212","7,210,913.562","114,977,252.342","36,611,133.412","4,873,009.776"
